In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

from sys import platform

if platform == "win32":
    path = 'C:/Users/olive/GitHub/f1-analytics/'
elif platform == "darwin":
    path = '~/Documents/GitHub/f1-analytics/'

In [ ]:
years = list(range(2014,  2022))

In [ ]:
def format_qualifying(x):
    if not any(i in x for i in ['DNF', 'DNS']):
        if ':' in x:
            return round(float(str(x).split(':')[1]) + (60 * float(str(x).split(':')[0])), 3) if x != 0 else 0
        else:
            return(round(float(x), 3))
    else:
        return x

In [ ]:
### Data collection method to scrape initial qualifiying results

qualifying_times = pd.DataFrame()
for year in years:
    url = 'https://www.formula1.com/en/results.html/{}/races.html'
    r = requests.get(url.format(year))
    soup = BeautifulSoup(r.text, 'html.parser')

    year_links = []
    for page in soup.find_all('a', attrs = {'class':"resultsarchive-filter-item-link FilterTrigger"}):
        link = page.get('href')
        if f'/en/results.html/{year}/races/' in link: 
            year_links.append(link)

    year_df = pd.DataFrame()
    new_url = 'https://www.formula1.com{}'

    for n, link in list(enumerate(year_links)):
        link = link.replace('race-result.html', 'qualifying.html')
        df = pd.read_html(new_url.format(link))
        df = df[0]
        df['season'] = year
        df['round'] = n+1

        for col in df:
            if 'Unnamed' in col:
                df.drop(col, axis = 1, inplace = True)

        year_df = pd.concat([year_df, df])

    qualifying_times = pd.concat([qualifying_times, year_df])

In [ ]:
qualifying_times.rename(columns = {'Pos': 'pos', 'Driver': 'driver', 'Car': 'constructor', 'Q1': 'q1', 'Q2': 'q2', 'Q3': 'q3'}, inplace=True)

qualifying_times.drop(['No', 'Laps'], axis=1, inplace=True)

qualifying_times.head()

In [ ]:
qualifying_times.query('season == 2016 & round == 3 & q3.isnull()')

In [ ]:
qualifying_times[['q1','q2','q3']] = qualifying_times[['q1','q2','q3']].applymap(lambda x: format_qualifying(str(x)))

qualifying_times.head()

In [ ]:
### Final data collection method to scrape final qualifying results

qualifying_results = pd.DataFrame()
for year in years:
    url = 'https://www.formula1.com/en/results.html/{}/races.html'
    r = requests.get(url.format(year))
    soup = BeautifulSoup(r.text, 'html.parser')

    year_links = []
    for page in soup.find_all('a', attrs = {'class':"resultsarchive-filter-item-link FilterTrigger"}):
        link = page.get('href')
        if f'/en/results.html/{year}/races/' in link: 
            year_links.append(link)

    year_df = pd.DataFrame()
    new_url = 'https://www.formula1.com{}'
    for n, link in list(enumerate(year_links)):
        link = link.replace('race-result.html', 'starting-grid.html')
        df = pd.read_html(new_url.format(link))
        df = df[0]
        df['season'] = year
        df['round'] = n+1
        for col in df:
            if 'Unnamed' in col:
                df.drop(col, axis = 1, inplace = True)

        year_df = pd.concat([year_df, df])

    qualifying_results = pd.concat([qualifying_results, year_df])

In [ ]:
qualifying_results.rename(columns = {'Pos': 'grid', 'Driver': 'driver', 'Car': 'constructor', 'Time': 'final_time'}, inplace=True)

qualifying_results.drop('No', axis=1, inplace=True)

qualifying_results.head()

In [ ]:
qualifying_results['final_time'] = qualifying_results.final_time.map(lambda x: format_qualifying(str(x)))

qualifying_results['final_time'].describe()

In [ ]:
# calculate difference in qualifying times

qualifying_results = qualifying_results[qualifying_results['final_time'] != 0]
qualifying_results.sort_values(['season', 'round', 'grid'], inplace = True)
qualifying_results['time_difference'] = qualifying_results.groupby(['season', 'round']).final_time.diff()
qualifying_results['q_delta'] = qualifying_results.groupby(['season', 'round']).time_difference.cumsum().fillna(0)
qualifying_results.drop('time_difference', axis=1, inplace=True)

qualifying_results.head()

In [ ]:
qualy_df = pd.merge(qualifying_results, qualifying_times, how='inner', on=['season', 'round', 'driver', 'constructor'])

qualy_df = qualy_df[['grid', 'pos', 'season', 'round', 'driver', 'constructor', 'final_time', 'q_delta', 'q1', 'q2', 'q3']]

qualy_df.query('season == 2020 & round == 1')

In [ ]:
qualy_df.to_csv(path+'data/qualifying.csv', index = False)